# MNIST training with MXNet and Gluon

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). This tutorial will show how to train and test an MNIST model on SageMaker using MXNet and the Gluon API.



In [1]:
import os
import sagemaker
from sagemaker.mxnet import MXNet
from sagemaker import get_execution_role

sess = sagemaker.Session()

role = get_execution_role()

output_path='s3://' + sess.default_bucket() + '/mxnet/mnist'

## MXNet Estimator

The `MXNet` class allows you to run your training script on SageMaker
infrastracture in a containerized environment. In this notebook, we will
refer to this container as *training container*. 

You need to configure
it with the following parameters to set up the environment:

- entry_point: A user defined python file to be used by the training container as the 
instructions for training. We will further discuss this file in the next subsection.

- role: An IAM role to make AWS service requests

- instance_type: The type of SageMaker instance to run your training script. 
Set it to `local` if you want to run the training job on 
the SageMaker instance you are using to run this notebook

- instance count: The number of instances you need to run your training job. 
Multiple instances are needed for distributed training.

- output_path: 
S3 location to save training output (model artifacts and output files)

- framework_version: The version of MXNet you need to use.

- py_version: The python version you need to use

For more information, see [the API reference](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase)



## Implement the entry point for training

The entry point for training is a python script that provides all 
the code for training a MXNet model. It is used by the SageMaker 
MXNet Estimator (`MXNet` class above) as the entry point for running the training job.

Under the hood, SageMaker MXNet Estimator downloads a docker image
with runtime environemnts 
specified by the parameters you used to initiated the
estimator class and it injects the training script into the 
docker image to be used as the entry point to run the container.

In the rest of the notebook, we use *training image* to refer to the 
docker image specified by the MXNet Estimator and *training container*
to refer to the container that runs the training image. 

This means your training script is very similar to a training script
you might run outside Amazon SageMaker, but it can access the useful environment 
variables provided by the training image. Checkout [the short list of environment variables provided by the SageMaker service](https://sagemaker.readthedocs.io/en/stable/frameworks/mxnet/using_mxnet.html?highlight=entry%20point) to see some common environment 
variables you might used. Checkout [the complete list of environment variables](https://github.com/aws/sagemaker-training-toolkit/blob/master/ENVIRONMENT_VARIABLES.md) for a complete 
description of all environment variables your training script
can access to. 

In this example, we will use the training script `code/train.py`
as the entry point for our MXNet Estimator.

The script here is an adaptation of the [Gluon MNIST example](https://github.com/apache/incubator-mxnet/blob/master/example/gluon/mnist.py) provided by the [Apache MXNet](https://mxnet.incubator.apache.org/) project. 

In [2]:
!cat 'code/train.py'

from __future__ import print_function

import argparse
import gzip
import logging
import os

import mxnet as mx
from mxnet import gluon, autograd
from mxnet.gluon import nn
from mxnet.gluon.data import Dataset
import numpy as np
import json
import time


logging.basicConfig(level=logging.DEBUG)

# ------------------------------------------------------------ #
# Training methods                                             #
# ------------------------------------------------------------ #

def input_transformer(data, label):
    data = data.reshape((-1,)).astype(np.float32) / 255.
    return data, label


class MNIST(Dataset):
    def __init__(self, data_dir, train=True, transform=None):

        if train:
            images_file="train-images-idx3-ubyte.gz"
            labels_file="train-labels-idx1-ubyte.gz"
        else:
            images_files="t10k-images-idx3-ubyte.gz"
            labels_files="t10k-labels-idx1-ubyte.gz"
        
        self.images, self.labels = self._convert_to

### Set hyperparameters

In addition, MXNet estimator allows you to parse command line arguments
to your training script via `hyperparameters`

In [3]:
# set local_mode to be True if you want to run the training script
# on the machine that runs this notebook

local_mode=False

if local_mode:
    instance_type='local'
else:
    instance_type='ml.c4.xlarge'
    
est = MXNet(
    entry_point='train.py',
    source_dir='code', # directory of your training script
    role=role,
    framework_version='1.7.0',
    py_version='py3',
    instance_type=instance_type,
    instance_count=1,
    output_path=output_path,
    hyperparameters={
        'batch-size':100,
        'epochs':1,
        'learning-rate': 0.1,
        'momentum': 0.9,
        'log-interval':100
    }
)


The training container executes your training script like

```
python train.py --batch-size 100 --epochs 20 --learning-rate 0.1
    --momentum 0.9 --log-interval 100
```

## Set up channels for training and testing data

You need to tell `MXNet` estimator where to find your training and 
testing data. It can be a link to an S3 bucket or it can be a path
in your local file system if you use local mode. In this example,
we use a public S3 bucket.

In [4]:
region_name=sess.boto_region_name

train_data_loc='s3://sagemaker-sample-files/datasets/image/MNIST'
test_data_loc='s3://sagemaker-sample-files/datasets/image/MNIST'

channels={
    'training': train_data_loc,
    'testing': test_data_loc
}

The keys of the dictionary `channels` are parsed to the training image
and it creates the environment variable `SM_CHANNEL_<key name>`. 

In this example, `SM_CHANNEL_TRAINING` and `SM_CHANNEL_TESTING` are created in the training image (checkout 
how `code/train.py` access these variables). For more information,
see: [SM_CHANNEL_{channel_name}](https://github.com/aws/sagemaker-training-toolkit/blob/master/ENVIRONMENT_VARIABLES.md#sm_channel_channel_name)

If you want, you can create a channel for validation:
```
channels = {
    'training': train_data_loc,
    'validation': val_data_loc,
    'test': test_data_loc
    }
```
You can then access this channel within your training script via
`SM_CHANNEL_VALIDATION`


## Run the training script on SageMaker
Now, the training container has everything to execute your training
script. You can start the container by calling `fit` method.

In [5]:
est.fit(inputs=channels)

ClientError: An error occurred (ValidationException) when calling the CreateTrainingJob operation: No S3 objects found under S3 URL "s3://sagemaker-sample-files/datasets/image/MNIST" given in input data source. Please ensure that the bucket exists in the selected region (us-west-2), that objects exist under that S3 prefix, and that the role "arn:aws:iam::688520471316:role/RL" has "s3:ListBucket" permissions on bucket "sagemaker-sample-files". Error message from S3: The bucket is in this region: us-east-1. Please use this region to retry the request

## Inspect and store model data

Now, the training is finished, the model artifact has been saved in 
the `output_path`. We 

In [ ]:
model_data = est.model_data
print("Model artifact saved at:\n", model_data)

We will store the variable `model_data` in the current notebook kernel. 
In the [next notebook](get_started_with_mnist_deploy.ipynb), you will learn how to retrieve the model artifact and deploy to a SageMaker
Endpoint.

In [ ]:
%store model_data

## Test and debug the entry point before executing the training container

The entry point `code/train.py` provided here has been tested and it can be executed in the training container. 
When you do develop your own training script, it is a good practice to simulate the container environment 
in the local shell and test it before sending it to SageMaker, because debugging in a containerized environment
is rather cumbersome. The following script shows how you can test your training script:

In [ ]:
!pygmentize code/test_train.py